In [9]:
import re
import csv
from collections import defaultdict, Counter
import os

In [11]:
os.mkdir(r"C:\Users\91912\OneDrive\Documents\Assignement\Sample.log")

In [15]:
# Configurable threshold for suspicious activity
FAILED_LOGIN_THRESHOLD = 10

# File paths
LOG_FILE = r"C:\Users\91912\OneDrive\Documents\Assignement\Sample.log"
OUTPUT_CSV = "log_analysis_results.csv"

def parse_log_file(file_path):
    """Parse the log file and return extracted data."""
    ip_requests = Counter()
    endpoint_requests = Counter()
    failed_logins = defaultdict(int)

    # Regular expressions to parse log entries
    log_pattern = r'(?P<ip>\d+\.\d+\.\d+\.\d+) - - \[.*?\] "(?P<method>\w+) (?P<endpoint>[^\s]+).*?" (?P<status>\d{3})'
    failed_login_pattern = r'"POST /login.*?" 401'

    with open(file_path, 'r') as file:
        for line in file:
            match = re.match(log_pattern, line)
            if match:
                ip = match.group("ip")
                endpoint = match.group("endpoint")
                status = match.group("status")

                # Count requests per IP
                ip_requests[ip] += 1

                # Count endpoint accesses
                endpoint_requests[endpoint] += 1

                # Identify failed login attempts
                if re.search(failed_login_pattern, line):
                    failed_logins[ip] += 1

    return ip_requests, endpoint_requests, failed_logins

def save_to_csv(ip_requests, most_accessed_endpoint, suspicious_ips, output_file):
    """Save analysis results to a CSV file."""
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        # Write requests per IP
        writer.writerow(["Requests per IP"])
        writer.writerow(["IP Address", "Request Count"])
        for ip, count in ip_requests.items():
            writer.writerow([ip, count])
        writer.writerow([])

        # Write most accessed endpoint
        writer.writerow(["Most Accessed Endpoint"])
        writer.writerow(["Endpoint", "Access Count"])
        writer.writerow(most_accessed_endpoint)
        writer.writerow([])

        # Write suspicious activity
        writer.writerow(["Suspicious Activity"])
        writer.writerow(["IP Address", "Failed Login Count"])
        for ip, count in suspicious_ips.items():
            writer.writerow([ip, count])

def main():
    # Parse the log file
    ip_requests, endpoint_requests, failed_logins = parse_log_file(LOG_FILE)

    # Get the most accessed endpoint
    most_accessed_endpoint = endpoint_requests.most_common(1)[0]

    # Filter suspicious IPs
    suspicious_ips = {ip: count for ip, count in failed_logins.items() if count > FAILED_LOGIN_THRESHOLD}

    # Display results
    print("Requests per IP:")
    for ip, count in ip_requests.items():
        print(f"{ip: <20} {count}")

    print("\nMost Frequently Accessed Endpoint:")
    print(f"{most_accessed_endpoint[0]} (Accessed {most_accessed_endpoint[1]} times)")

    print("\nSuspicious Activity Detected:")
    if suspicious_ips:
        for ip, count in suspicious_ips.items():
            print(f"{ip: <20} {count}")
    else:
        print("No suspicious activity detected.")

    # Save results to CSV
    save_to_csv(ip_requests, most_accessed_endpoint, suspicious_ips, OUTPUT_CSV)
    print(f"\nResults saved to {OUTPUT_CSV}")

if __name__ == "__main__":
    main()


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\91912\\OneDrive\\Documents\\Assignement\\Sample.log'